In [2]:
import pandas as pd

## Предварительная обработка данных

In [27]:
df = pd.read_csv('bag_data.csv')
print(df.shape)
df.head()

(1063, 16)


,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Наличие пыльника,Состояние товара,Продавец,Oskelly ID,Цена,Модель,Винтаж,Наличие коробки,Наличие сертификата
0,INT L,Женское,Сумки с короткими ручками,MARC JACOBS,Экзотическая кожа,Серебряный,Средние ручки,Да,Отличное состояние,Частный продавец,1993293,52000,NaN,NaN,NaN,NaN
1,INT S,Женское,Клатчи и вечерние сумки,VERSACE,Кожа,Черный,Короткие ручки,NaN,Отличное состояние,Частный продавец,1993444,38000,NaN,NaN,NaN,NaN
2,INT M,Женское,Сумки через плечо,BALENCIAGA,Кожа,Желтый,Длинный ремень,NaN,Новое с биркой,Частный продавец,2035063,345000,Crush,NaN,NaN,NaN
3,INT U,Женское,Сумки с короткими ручками,BURBERRY,Кожа,Коричневый,Средние ручки,NaN,Отличное состояние,Частный продавец,1941649,27000,NaN,Да,NaN,NaN
4,INT M,Женское,Сумки с короткими ручками,KARL LAGERFELD,Искусственная кожа,Мульти,Средние ручки,NaN,Новое с биркой,Частный продавец,2008104,17950,NaN,NaN,NaN,NaN


Подготовленный мной датасет представляет из себя 1063 объявления, опубликованных на сайте oskelly.com, специализирующемся на вторичной продаже сумок. Каждое объявление содержит информацию о продаваемой сумке и ее стоимость.

## Анализ признаков

Итак, исходный датасет имеет 15 признаков/колонок и 1 целевую переменную, которую мы в финальной части нашего проекта будем предсказывать. Это переменная 'Цена'. Теперь давайте пройдемся по каждому столбцу нашей таблицы и посмотрим, что в них содержится:
   * **Размер** - размер сумки
       * INT U - ONE SIZE
       * INT XS 
       * INT S
       * INT M
       * INT L
       * INT XL       
   * **Раздел** - Женское/Мужское
   * **Категории** - тоут/через плечо/с короткими ручками/рюкзаки/клатчи/аксессуары для сумок
   * **Бренд** - бренд сумки 
   * **Материал** - материал сумок 
   * **Цвет** - цвет сумки
   * **Длина ручки** 
   * **Наличие пыльника**
   * **Состояние товара** - отличное/хорошее/новое с биркой 
   * **Продавец** - частный продавец/бутик 
   * **Oskelly ID** - ID продавца
   * **Модель** - если у бренда есть определенная модель сумок 
   * **Винтаж** - является ли сумка винтажной 
   * **Наличие коробки**
   * **Наличие сертификата** - наличие сертификата подлинности

In [28]:
df.isna().sum()

Размер                   3
Раздел                   0
Категория                0
Бренд                    0
Материал сумок           0
Цвет                     0
Длина ручки              0
Наличие пыльника       751
Состояние товара         0
Продавец                 0
Oskelly ID               0
Цена                     0
Модель                 664
Винтаж                 992
Наличие коробки        965
Наличие сертификата    973
dtype: int64

In [29]:
missing_size = df.loc[df['Размер'].isnull()]
print(missing_size)

    Размер   Раздел   Категория        Бренд Материал сумок     Цвет  \
500    NaN  Женское  Сумки тоут        PRADA           Кожа  Бежевый   
754    NaN  Женское  Сумки тоут        PRADA           Кожа  Бежевый   
874    NaN  Женское  Сумки тоут  MARC JACOBS           Кожа   Черный   

        Длина ручки Наличие пыльника   Состояние товара          Продавец  \
500   Средние ручки              NaN  Хорошее состояние  Частный продавец   
754   Средние ручки              NaN  Хорошее состояние  Частный продавец   
874  Длинный ремень              NaN     Новое с биркой  Частный продавец   

     Oskelly ID   Цена        Модель Винтаж Наличие коробки  \
500     2008263  30500           NaN     Да             NaN   
754     2008263  30500           NaN     Да             NaN   
874     1926273  62000  The Tote Bag    NaN             NaN   

    Наличие сертификата  
500                 NaN  
754                 NaN  
874                 NaN  


Заменим в строках с пропущенным размером на INT U

In [30]:
df['Размер'] = df['Размер'].fillna('INT U')

In [31]:
unique_values = df['Oskelly ID'].nunique()

print("Количество уникальных значений в колонке 'Oskelly ID':", unique_values)

Количество уникальных значений в колонке 'Oskelly ID': 810


Это достаточно уникальный признак для каждой строки, тем более мы анализируем стоимость сумок впринципе, поэтому удалим этот признак. Признак "Модель" оставим, так как он заполнен для действительно культовых классических моделей, стоимость которых строится не только на материале, размере и имени бренда, но во многом на самой модели. Пустые значения в этой колонке заменим на "Ordinary"

In [32]:
df.drop(columns=['Oskelly ID'], inplace=True)
df['Модель'] = df['Модель'].fillna('Ordinary')

Бинарные признаки приведем к необходимому виду, где 1 соответсвует "Да", а 0 - пустой ячейке

In [33]:
df['Винтаж'] = df['Винтаж'].apply(lambda x: 1 if x == 'Да' else 0)
df['Наличие пыльника'] = df['Наличие пыльника'].apply(lambda x: 1 if x == 'Да' else 0)
df['Наличие коробки'] = df['Наличие коробки'].apply(lambda x: 1 if x == 'Да' else 0)
df['Наличие сертификата'] = df['Наличие сертификата'].apply(lambda x: 1 if x == 'Да' else 0)

In [34]:
df.isna().sum()

Размер                 0
Раздел                 0
Категория              0
Бренд                  0
Материал сумок         0
Цвет                   0
Длина ручки            0
Наличие пыльника       0
Состояние товара       0
Продавец               0
Цена                   0
Модель                 0
Винтаж                 0
Наличие коробки        0
Наличие сертификата    0
dtype: int64

In [35]:
df.head()

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Наличие пыльника,Состояние товара,Продавец,Цена,Модель,Винтаж,Наличие коробки,Наличие сертификата
0,INT L,Женское,Сумки с короткими ручками,MARC JACOBS,Экзотическая кожа,Серебряный,Средние ручки,1,Отличное состояние,Частный продавец,52000,Ordinary,0,0,0
1,INT S,Женское,Клатчи и вечерние сумки,VERSACE,Кожа,Черный,Короткие ручки,0,Отличное состояние,Частный продавец,38000,Ordinary,0,0,0
2,INT M,Женское,Сумки через плечо,BALENCIAGA,Кожа,Желтый,Длинный ремень,0,Новое с биркой,Частный продавец,345000,Crush,0,0,0
3,INT U,Женское,Сумки с короткими ручками,BURBERRY,Кожа,Коричневый,Средние ручки,0,Отличное состояние,Частный продавец,27000,Ordinary,1,0,0
4,INT M,Женское,Сумки с короткими ручками,KARL LAGERFELD,Искусственная кожа,Мульти,Средние ручки,0,Новое с биркой,Частный продавец,17950,Ordinary,0,0,0


In [38]:
# Сохраним итоговую таблицу
df.to_csv('bags_df_process.csv', index=False)